In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Project imports
import pyhalomodel.pyhalomodel as halo
import pyhalomodel.camb_stuff as camb_stuff
import pyhalomodel.utility as util
from pyhalomodel.pyhmcode import hmcode as HMcode

In [ ]:
# Cosmology
#Omega_c = 0.2283
Omega_c = 0.2435
#Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
ns = 0.96
sigma_8 = 0.8
w0 = -1.
wa = 0.
#m_nu = 1.00
m_nu = 0.30
#m_nu = 0.00

# Wavenumbers [h/Mpc]
kmin, kmax = 1e-3, 1e1
nk = 128
k = util.logspace(kmin, kmax, nk)

# Halo masses [Msun/h]
Mmin, Mmax = 1e9, 1e17
nM = 256
M = util.logspace(Mmin, Mmax, nM)

# Redshifts
zs = [3., 2., 1., 0.5, 0.]
zs = np.array(zs)

In [ ]:
# Get stuff from CAMB
Pk_lin_interp, results, Omega_m, _, _ = camb_stuff.run(zs, Omega_c, Omega_b, Omega_k, h, ns, sigma_8, m_nu, w0, wa)
Pk_nonlin_interp = results.get_matter_power_interpolator(nonlinear=True).P

# Arrays for CAMB non-linear spectrum
Pk_CAMB = np.zeros((len(zs), len(k)))
for iz, z in enumerate(zs):
    Pk_CAMB[iz, :] = Pk_nonlin_interp(z, k)

In [ ]:
# Get the new pyHMcode spectrum
Pk_HMcode = HMcode(k, zs, results, verbose=True)

In [ ]:
# Halo model spectrum
Pk_hm = []
for iz, z in enumerate(zs):

    # Initialize
    hmod = halo.model(z, Omega_m, name='Tinker et al. (2010)', Dv=200., dc=1.686)

    # Halo profiles
    R = hmod.Lagrangian_radius(M)
    rv = hmod.virial_radius(M)
    c = halo.concentration(M, z)
    matter_profile = halo.matter_profile(k, M, rv, c, Omega_m)

    # Power spectra
    Pk_lin = Pk_lin_interp(z, k)
    sigmaM = results.get_sigmaR(R, z_indices=[iz])[0]
    _, _, Pk = hmod.power_spectrum(k, Pk_lin, M, sigmaM, {'m': matter_profile})
    Pk_hm.append(Pk['m-m'])

In [ ]:
# Initialize
plt.subplots(2, 1)
dr = 15.
#dr = 1.5
#dr = 0.15

# Power
plt.subplot(2, 1, 1)
plt.plot(np.nan, color='black', label='HMcode: Python')
plt.plot(np.nan, color='black', alpha=0.5, label='HMcode: CAMB')
plt.plot(np.nan, color='black', ls=':', label='Halo model')
for iz, z in enumerate(zs):
    plt.loglog(k, Pk_HMcode[iz, :], color='C%d'%iz, label='z = %1.1f'%z)
    plt.loglog(k, Pk_hm[iz], ls=':', color='C%d'%iz)
    plt.loglog(k, Pk_CAMB[iz, :], alpha=0.5, color='C%d'%iz)
plt.xticks([])
plt.ylabel('P(k)')
plt.legend(ncol=2)

# Ratio
plt.subplot(2, 1, 2)
plt.axhspan(-1., 1., color='black', alpha=0.2)
plt.axhspan(-0.1, 0.1, color='black', alpha=0.2)
plt.axhline(0., color='black')
for iz, _ in enumerate(zs):
    plt.semilogx(k, 100.*(-1.+Pk_hm[iz]/Pk_CAMB[iz, :]), ls=':', color='C%d'%iz)
    plt.semilogx(k, 100.*(-1.+Pk_HMcode[iz, :]/Pk_CAMB[iz, :]), color='C%d'%iz)
plt.xlabel('$k/h\mathrm{Mpc}^{-1}$')
plt.ylabel('$P(k)/P_\mathrm{CAMB}(k)-1$ [%]')
plt.ylim((-dr, dr))

# Finalize
plt.tight_layout()
plt.show()